In [1]:
import random
from itertools import cycle

# Constants
# Maps int to string for bid quantity
quantity_words = dict(zip([i for i in range(1, 11)], ["One", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine", "Ten"]))
dice_words = dict(zip([i for i in range(1, 7)], ["ones", "twos", "threes", "fours", "fives", "sixes"])) # Maps int to string for bid value

In [2]:
class Die:
    """ Represents a die object."""
    def __init__(self, value=None):
        self.value = value

    def __repr__(self):
        return str(self.value)

    def roll(self):
        self.value = random.randint(1, 6)

class Player:
    """ Represents a player in the game."""
    def __init__(self, name):
        self.name = name
        self.dice = [Die() for i in range(5)]

    def __repr__(self):
        return_string = ""
        return_string += str("Name: ") + self.name + " "
        return_string += str([d.value for d in self.dice])
        return return_string

    def get_name(self):
        return self.name
    
    def get_dice(self):
        return self.dice

    def roll(self):
        for die in self.dice:
            die.roll()

    def lose_die(self):
        """When a player loses a round they lose a die"""
        self.dice.pop()

    def gain_die(self):
        """When a player wins an ExactCall they win a die"""
        self.dice.append(Die())

    def bid(self, previous_bid=None):
        # Maybe a try/except to stop erroneous bids (e.g. someone bid's a string etc)
        
        # Create bid
        quantity = int(input("Bid quantity: "))
        value = int(input("Bid value: "))
        bid = Bid(quantity, value)

        # Beginning of a bidding round, no previous bid
        if previous_bid is None :
            return bid
        if bid > previous_bid : 
            return bid
        else : 
            # Bid was too low- make another bid.
            print("Your bid must be higher than the previous bid!")
            return self.bid(previous_bid)

    def call(self, previous_bid):
        """Response to a bid where you think that there won't be that many dice rolled. e.g. Calling bullshit! This ends a round"""
        pass # To implement

    def has_lost(self):
        return not(self.dice)

    def is_remaining(self):
        return bool(self.dice)

class Bid:
    """ Represents a Bid in the game."""
    def __init__(self, quantity, value) :
        self.quantity = quantity # {1 -> n} where n is number of dice in play
        self.value = value # {1, 2, 3, 4, 5}
        pass

    def __repr__(self) :
        return str(f"{quantity_words[self.quantity]} {dice_words[self.value]}")

    def __eq__(self, other) :
        return self.quantity == other.quantity and self.value == other.value

    def __gt__(self, other) :
        if self.not_ace_bid() and other.not_ace_bid() :
            if self.quantity == other.quantity :
                return self.value > other.value
            else :
                return self.quantity > other.quantity
        if self.is_ace_bid() and other.not_ace_bid() : 
            return self.quantity * 2 >= other.quantity
        if self.not_ace_bid() and other.is_ace_bid() : 
            return self.quantity > other.quantity * 2
        if self.is_ace_bid() and other.is_ace_bid() :
            return self.quantity > other.quantity

    def __lt__(self, other):
        if self.not_ace_bid() and other.not_ace_bid() : 
            if self.quantity == other.quantity :
                return self.value < other.value
            else :
                return self.quantity < other.quantity
        if self.is_ace_bid() and other.not_ace_bid() : 
            return self.quantity * 2 <= other.quantity
        if self.not_ace_bid() and other.is_ace_bid() :
            return self.quantity < other.quantity * 2
        if self.is_ace_bid() and other.is_ace_bid() :
            return self.quantity < other.quantity

    def is_ace_bid(self):
        return self.value == 1

    def not_ace_bid(self):
        return self.value != 1

class Game():
    """ Represents an actual instance of the game Liar's Dice (Dudo)."""
    def __init__(self):
        self.players = self.generate_players()
        self.player_cycle = cycle(self.players)
        self.first_to_act = self.get_next_player() # Helps get cycle object back to correct place after a round.

    def __repr__(self):
        return_string = ""
        for player in self.players:
            return_string += str(player) + "\n"
        return return_string
        
    def generate_players(self):
        num_players = int(input("How many players are playing?"))
        players = []
        
        # Create the Player objects and store them inside players list
        for i in range(num_players):
            name = input("Player's name?")
            player = Player(name)
            players.append(player)
        return players

    def is_finished(self):
        """When less than 2 players are remaining, i.e only 1 player is left, then they have won and the game is over!"""
        return self.players_remaining() < 2

    def get_players(self):
        return self.players

    def get_first_to_act(self):
        return self.first_to_act

    def players_remaining(self):
        """Returns the number of players in the game that still have dice (and still have a chance to win)."""
        num_remaining = 0
        for player in self.players:
            num_remaining += int(player.is_remaining())
        return num_remaining

    def get_next_player(self):
        next_player = next(self.player_cycle)
        if next_player.is_remaining() : 
            return next_player
        else :
            return self.get_next_player()

    def all_players_roll_dice(self):
        for player in self.players:
            player.roll()


    def ensure_cycle_correct(self, bidder):
        print("is it my fault?? Sorry. :(")
        next_player = next(self.player_cycle)
        if next_player is bidder : # Possibly should use == and implement a __eq__ for Player class... ?
            return
        else :
            self.ensure_cycle_correct(next_player)

    def play_round(self) : 
        print("Playing Round!")
        self.all_players_roll_dice()
        bidder = self.first_to_act # The loser of the last round is the first to act.
        print(f"{bidder.get_name()} must make a bid!") # Round must start with a bid.
        print("Just about to make a bid.")
        bid = bidder.bid()
        print("just made a bid.")

        # Cycle could be in the wrong place and get_next_player() not necessarily going to get the correct player!!
        print("Did I make it here?! (1)")
        
        #self.ensure_cycle_correct(bidder=bidder) # This iss breaking the code

        responder = self.get_next_player() # One to the left of the first bidder.
        
        # Response to a bid can be either: Bid, Call, or ExactCall.
        response = self.faceoff(bidder, bid, responder)

        # Repeatedly have faceoffs until a non-Bid response.
        while isinstance(response, Bid):
            bidder, responder = responder, self.get_next_player()
            response = self.faceoff(bidder, response, responder)

        if isinstance(response, Call):
            call = response
            self.check_results(bid, call)
            pass

        if isinstance(response, ExactCall):
            exact_call = response
            self.check_results(bid, exact_call)
            pass

        # Shouldn't ever reach here....

    def faceoff(self, bidder, bid, responder):
        """Returns a response to bidder's bid. Returns either a (higher) Bid, a Call, or an ExactBid."""
        print(f"{bidder.get_name()} has made a bid of: {bid}.")
        print(f"{responder.get_name()} is responding to {bidder.get_name()}'s bid...")

        response_string = self.get_bid_response()

        # Bid response
        if response_string == "Bid":
            response_bid = responder.bid(previous_bid=bid)
            print(f"{responder.get_name()} responds with a bid of: {response_bid}")
            return response_bid

        # Call response
        if response_string == "Call" : return


        # ExactCall response
        if response_string == "ExactCall" : return

    def get_bid_response(self):
        while True:
            response_string = input("Response: ")
            if response_string in ["Bid", "Call", "ExactCall"]:
                return response_string
            else :
                print("Invalid response!")
                continue


class Call():
    """
    Represents when a player calls a bid- ending that bidding round and starting the comparisons.
    """
    def __init__(self):
        pass

class ExactCall():
    """Represents when a player makes an ExactCall response- a special type of Call"""

In [ ]:
# To do list


# Quantity words dict only goes up to 10 so a bid of 11 or more will break the game.
# Ensure logic of lt and gt for bids is correct (Write tests????)
# Set up the Game class which is going to be an instance of an actual game of Liar's dice
# Test equals case for gt and lt

# Game class

# Fields ? 
# players field which stores the Player objects (list of Player() objects)
# players_cycle which cycles through the players remaining
# game_finished - boolean True/False

# write the method for playing a round.

# rounds are made of of a series of bids which go higher and higher.
# round can perhaps be broken down into smaller actions, which is basically first a bid and then second a response.
# a response can be a higher bid, a call, or an exact call.
# possibly I can make a class such as "Action" which is a super class of Bid, Call, and ExactCall?
# then logic is response = player.get_response()
# if response isinstance(Bid) -> then re-do.
# elif response isinstance(Call) -> initiate call process and end the round.
# else (response must be ExactCall) -> initiate ExactCall process and end the round.

# Things to keep in mind-

# Cannot simply just say "get next player" all the time because the player to begin a round will change depending on if a call was successful or not.
# Should make a field in the __init__ method called "first_to_act" and then update this player depending on what happened at the end of the round.
# this means need to keep a track of not only the Bid and if it was successful or not, but the player that made the bid.
# Possible a bid object needs to have an "owner"- ie the person that made the bid?


# is_finished()


# Bid class

# Create a method called value() which would simplify the logic in the __lt__ and __gt__ methods.


# Flow of the game is something like....

# Set-up / Initialisation:

# Set up all players, each starts with 6 die objects
# Randomly select a player to start

# Start of game :
# Have a round
# Have the loser of the round lose a die object
# Have the winner of the round gain a die (if relevant)
# Set up the first_to_act player depending on what happened in the round.


# Game continues : 
# Have another round... etc (repeat above)
# Continue having rounds until the game is finished. e.g.
# while game.not_finished() :
#     play_round()

# Game finished (just after the while loop logic)
# print the winners name???



# Can expand the game by changing how the ordering of players is deciding, by rolling dice (see the wikipedia page for rules)

# Make it so bid values can only be between 1 and 6
# Make it so bid quantities can only be between 1 and num_dice_remaining

# Should ExactCall class inherit from Call?